In [ ]:
>>> import textstat

>>> test_data = (
    "Playing games has always been thought to be important to "
    "the development of well-balanced and creative children; "
    "however, what part, if any, they should play in the lives "
    "of adults has never been researched that deeply. I believe "
    "that playing games is every bit as important for adults "
    "as for children. Not only is taking time out to play games "
    "with our children and other adults valuable to building "
    "interpersonal relationships but is also a wonderful way "
    "to release built up tension."
)

>>> textstat.flesch_reading_ease(test_data)
>>> textstat.smog_index(test_data)
>>> textstat.flesch_kincaid_grade(test_data)
>>> textstat.coleman_liau_index(test_data)
>>> textstat.automated_readability_index(test_data)
>>> textstat.dale_chall_readability_score(test_data)
>>> textstat.difficult_words(test_data)
>>> textstat.linsear_write_formula(test_data)
>>> textstat.gunning_fog(test_data)
>>> textstat.text_standard(test_data)

In [1]:
test_data = (
    "Playing games has always been thought to be important to "
    "the development of well-balanced and creative children; "
    "however, what part, if any, they should play in the lives "
    "of adults has never been researched that deeply. I believe "
    "that playing games is every bit as important for adults "
    "as for children. Not only is taking time out to play games "
    "with our children and other adults valuable to building "
    "interpersonal relationships but is also a wonderful way "
    "to release built up tension."
)

In [21]:
textstat.flesch_reading_ease(test_data)

52.23

In [1]:
import textstat

In [2]:
from article_process_new import ArticleLM

In [3]:
# path_to_data = '/Users/stephanie/data/newsela_articles_corpus_2019-03-25_SMALL'
path_to_data = '/Users/stephanie/data/newsela_articles_corpus_2019-03-25'
path_to_kenlm = '/Users/stephanie/github/kenlm'
path_to_arpa = path_to_kenlm + '/lm'

In [5]:
articleLM = ArticleLM(path_to_data, path_to_kenlm, path_to_arpa, 5, 'grade_level')

In [6]:
# articleLM.metadata_split[articleLM.metadata_split.train_val_test=='val'].file_path

In [7]:
# filename = articleLM.metadata_split[articleLM.metadata_split.train_val_test=='val'].loc[9,'file_path']

In [8]:
textstat.flesch_reading_ease(articleLM.get_article_text(filename))

NameError: name 'filename' is not defined

In [18]:
# filename = articleLM.metadata_split.loc[9,articleLM.metadata_split.train_val_test=='val'].file_path
articleLM.get_article_text(filename)

'\ufeffIn 2017, the government of Ontario began a social experiment. The Canadian province is giving as much as $16,989 ($12,616 in U.S. dollars) a year to about 4,000 low-income people. It makes no difference whether they\'re working or on government assistance. Over the course of three years, Ontario will track the recipients to see if they have wasted the money or used it wisely. Ontario Premier Kathleen Wynne said she hopes they will "get ahead and stay ahead."\n\nThe experiment is similar to a program that began in Finland the same year. That plan gives about $7,200 annually to 2,000 unemployed people but takes them off normal government assistance. Even if they do find work, they keep getting the money.\n\nThese are two of the boldest attempts to test a bigger idea called universal basic income. It is when the government gives money to everyone each month regardless of how much money they have.\n\nThe idea has been proposed for centuries. Lately it has become popular among some l

In [9]:
article_scores = articleLM.metadata_split[['slug', 'language', 'grade_level', 'category', 'file_path',
                                          'train_val_test', 'is_original']].copy()

In [10]:
article_scores = article_scores[(article_scores.language=='en')
                               & (article_scores.grade_level != 10)
                               & (article_scores.is_original == False)]

In [11]:
article_scores.head()

,slug,language,grade_level,category,file_path,train_val_test,is_original
0,georgetown-slavery-admission,en,3,law,articles/georgetown-slavery-admission.en.03.txt,train,False
1,headphone-earbuds,en,6,health,articles/headphone-earbuds.en.06.txt,train,False
2,afghanistan-ustroops,en,7,war-and-peace,articles/afghanistan-ustroops.en.07.txt,train,False
3,remembering-historic-giant-tree,en,8,science,articles/remembering-historic-giant-tree.en.08...,train,False
5,riley-dog-veterans,en,5,health,articles/riley-dog-veterans.en.05.txt,train,False


In [19]:
def flesch(filename):
    return textstat.flesch_reading_ease(articleLM.get_article_text(filename))
def flesch_k_grade(filename):
    return textstat.flesch_kincaid_grade(articleLM.get_article_text(filename))
def automated_read(filename):
    return textstat.automated_readability_index(articleLM.get_article_text(filename))

In [20]:
def flesch_map(flesch_score):
    if flesch_score <= 29:
        return 'very_confusing'
    elif flesch_score >= 30 and flesch_score <=49:
        return 'difficult'
    elif flesch_score >= 50 and flesch_score <=59:
        return 'fairly_difficult'
    elif flesch_score >= 60 and flesch_score <=69:
        return 'standard'
    elif flesch_score >= 70 and flesch_score <=79:
        return 'fairly_easy'
    elif flesch_score >= 80 and flesch_score <=89:
        return 'easy'
    else:
        return 'very_easy'

In [21]:
article_scores.loc[:,'flesch'] = list(map(flesch, article_scores.file_path))
article_scores.loc[:,'flesch_mapping'] = list(map(flesch_map, article_scores.flesch))
article_scores.loc[:,'flesch_k_grade'] = list(map(flesch_k_grade, article_scores.file_path))
article_scores.loc[:,'automated_read'] = list(map(automated_read, article_scores.file_path))

In [22]:
article_scores.loc[:,'true_easy'] = [1 if x <= 5 else 0 for x in article_scores.grade_level]
article_scores.loc[:,'flesch_easy'] = [1 if x <= 79 else 0 for x in article_scores.flesch]

In [23]:
article_scores.loc[:,'flesch_k_grade'] = article_scores.flesch_k_grade.astype('int')
article_scores.loc[:,'automated_read'] = article_scores.automated_read.astype('int')
article_scores.loc[:,'flesch_kgrade_easy'] = [1 if x <= 5 else 0 for x in article_scores.flesch_k_grade]
article_scores.loc[:,'auto_read_easy'] = [1 if x <= 5 else 0 for x in article_scores.automated_read]

In [25]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

cm = confusion_matrix(article_scores.true_easy, article_scores.flesch_easy)
cm_fkg = confusion_matrix(article_scores.grade_level, article_scores.flesch_k_grade)
cm_fkg_easy = confusion_matrix(article_scores.true_easy, article_scores.flesch_kgrade_easy)
cm_autoread = confusion_matrix(article_scores.true_easy, article_scores.auto_read_easy)

In [26]:
cm

array([[   3, 8452],
       [4658, 6425]])

In [27]:
cm_autoread

array([[ 8453,     2],
       [10078,  1005]])

In [29]:
accuracy_score(article_scores.true_easy, article_scores.flesch_easy)

0.32899989763537724

In [30]:
accuracy_score(article_scores.grade_level, article_scores.flesch_k_grade)

0.04074111986897328

In [32]:
accuracy_score(article_scores.true_easy, article_scores.flesch_kgrade_easy)

0.6482751561060498

In [33]:
accuracy_score(article_scores.true_easy, article_scores.auto_read_easy)

0.48408230115672024

In [35]:
f1_score(article_scores.true_easy, article_scores.flesch_easy, pos_label=1)

0.4949922958397534

In [36]:
f1_score(article_scores.true_easy, article_scores.flesch_kgrade_easy, pos_label=1)

0.5507909530657602

In [37]:
f1_score(article_scores.true_easy, article_scores.auto_read_easy, pos_label=1)

0.1662531017369727

In [52]:
cm_fkga


array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  2,  62, 334, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]])

In [42]:
len(article_scores[article_scores.true_easy == article_scores.flesch_easy] ) * 100/ len(article_scores)

13.041252942982904

In [57]:
len(article_scores[article_scores.true_easy == article_scores.flesch_kgrade_easy] ) * 100/ len(article_scores)

64.82751561060498

In [66]:
len(article_scores[article_scores.true_easy == article_scores.auto_read_easy] ) * 100/ len(article_scores)

48.408230115672026